<a href="https://colab.research.google.com/github/oxygen0605/ImageClassification/blob/master/ResNet_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratory環境の初期設定

## Google Driveにマウントしてマシンスペックを出力

In [0]:
from google.colab import drive 
drive.mount('/content/drive')

!nvidia-smi > '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'
!cat /proc/driver/nvidia/gpus/0000:00:04.0/information >> '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'
!cat /etc/issue >> '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'
!cat /proc/cpuinfo >> '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls -al

total 20
drwxr-xr-x 1 root root 4096 Aug 26 14:36 .
drwxr-xr-x 1 root root 4096 Aug 26 13:24 ..
drwxr-xr-x 1 root root 4096 Aug 22 16:14 .config
drwx------ 3 root root 4096 Aug 26 14:36 drive
drwxr-xr-x 1 root root 4096 Aug 22 16:14 sample_data


# Deep CNN (CIFAR-10)

## モデルの生成

In [0]:
from functools import reduce
from keras import backend
from keras.layers import (Activation, Add, GlobalAveragePooling2D,BatchNormalization, Conv2D, Dense, Flatten, Input, MaxPooling2D)
from keras.models import Model
from keras.regularizers import l2

def compose(*funcs):
	'''
		複数の層を結合する。
	'''
	if funcs:
		return reduce(lambda f, g: lambda *args, **kwargs: g(f(*args, **kwargs)), funcs)
	else:
		raise ValueError('Composition of empty sequence not supported.')
		
def ResNetConv2D(*args, **kwargs):
	'''
		conv を作成する。
	'''
	conv_kwargs = {
			'strides': (1, 1),
			'padding': 'same',
			'kernel_initializer': 'he_normal',
			'kernel_regularizer': l2(1.e-4)
	}
	
	# update:辞書の要素を更新
	conv_kwargs.update(kwargs) 
	
	return Conv2D(*args, **conv_kwargs)
	
def bn_relu_conv(*args, **kwargs):
	'''
		batch mormalization -> ReLU -> conv を作成する。
	'''
	return compose(
		BatchNormalization(),
		Activation('relu'),
		ResNetConv2D(*args, **kwargs))

def shortcut(x, residual):
	'''
		shortcut connection を作成する。
	'''
	x_shape = backend.int_shape(x)
	residual_shape = backend.int_shape(residual)
	
	if x_shape == residual_shape:
		# x と residual の形状が同じ場合、なにもしない。
		shortcut = x
	else:
		# x と residual の形状が異なる場合、線形変換を行い、形状を一致させる。
		stride_w = int(round(x_shape[1] / residual_shape[1]))
		stride_h = int(round(x_shape[2] / residual_shape[2]))
		
		shortcut = Conv2D(filters=residual_shape[3],
					 kernel_size=(1, 1),
					 strides=(stride_w, stride_h),
					 kernel_initializer='he_normal',
					 kernel_regularizer=l2(1.e-4))(x)
	
	return Add()([shortcut, residual])

# ResNet-18, ResNet-34用
def basic_block(filters, first_strides, is_first_block_of_first_layer):
	'''bulding block を作成する。
	
        Arguments:
            filters: フィルター数
            first_strides: 最初の畳み込みのストライド
            is_first_block_of_first_layer: max pooling 直後の residual block かどうか
	'''
	def f(x):
		if is_first_block_of_first_layer:
			# conv1 で batch normalization -> ReLU はすでに適用済みなので、
			# max pooling の直後の residual block は畳み込みから始める。
			conv1 = ResNetConv2D(filters=filters, kernel_size=(3, 3))(x)
		else:
			conv1 = bn_relu_conv(filters=filters, kernel_size=(3, 3),
						strides=first_strides)(x)
			
		conv2 = bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
		
		return shortcut(x, conv2)

	return f

# ResNet-50, ResNet-101, ResNet-152用
def bottleneck_block(filters, first_strides, is_first_block_of_first_layer):
	'''bottleneck bulding block を作成する。
	
	        Arguments:
	            filters: フィルター数
	            first_strides: 最初の畳み込みのストライド
	            is_first_block_of_first_layer: max pooling 直後の residual block かどうか
	'''

	def f(x):
		if is_first_block_of_first_layer:
			# conv1 で batch normalization -> ReLU はすでに適用済みなので、
			# max pooling の直後の residual block は畳み込みから始める。
			conv1 = ResNetConv2D(filters=filters, kernel_size=(3, 3))(x)
		else:
			conv1 = bn_relu_conv(filters=filters, kernel_size=(1, 1),
						strides=first_strides)(x)
		
		conv2 = bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
		conv3 = bn_relu_conv(filters=filters * 4, kernel_size=(1, 1))(conv2)
		
		return shortcut(x, conv3)

	return f

# 複数の residual block を反復させるヘルパー関数
def residual_blocks(block_function, filters, repetitions, is_first_layer):
	'''residual block を反復する構造を作成する。

        Arguments:
            block_function: residual block を作成する関数
            filters: フィルター数
            repetitions: residual block を何個繰り返すか。
            is_first_layer: max pooling 直後かどうか
	'''
	def f(x):
		for i in range(repetitions):
			# conv3_x, conv4_x, conv5_x の最初の畳み込みは、
			# プーリング目的の畳み込みなので、strides を (2, 2) にする。
			# ただし、conv2_x の最初の畳み込みは直前の max pooling 層でプーリングしているので
			# strides を (1, 1) にする。
			first_strides = (2, 2) if i == 0 and not is_first_layer else (1, 1)
			
			x = block_function(filters=filters, first_strides=first_strides,
					  is_first_block_of_first_layer=(i == 0 and is_first_layer))(x)
		
		return x
	
	return f

class ResnetBuilder():
	@staticmethod
	def build(input_shape, num_outputs, block_type, repetitions):
		'''ResNet モデルを作成する Factory クラス
        Arguments:
            input_shape: 入力の形状
            num_outputs: ネットワークの出力数
            block_type : residual block の種類 ('basic' or 'bottleneck')
            repetitions: 同じ residual block を何個反復させるか
		'''
		# block_type に応じて、residual block を生成する関数を選択する。
		if block_type == 'basic':
			block_fn = basic_block
		elif block_type == 'bottleneck':
			block_fn = bottleneck_block

		# モデルを作成する。
		##############################################
		input = Input(shape=input_shape)

		# conv1 (batch normalization -> ReLU -> conv)
		conv1 = compose(ResNetConv2D(filters=64, kernel_size=(7, 7), strides=(2, 2)),
						BatchNormalization(),
						Activation('relu'))(input)

		# pool
		pool1 = MaxPooling2D(
				pool_size=(3, 3), strides=(2, 2), padding='same')(conv1)
		
		# conv2_x, conv3_x, conv4_x, conv5_x
		block = pool1
		filters = 64
		for i, r in enumerate(repetitions):
			block = residual_blocks(block_fn, filters=filters, repetitions=r,
									is_first_layer=(i == 0))(block)
			filters *= 2

		# batch normalization -> ReLU
		block = compose(BatchNormalization(),
						Activation('relu'))(block)

		# global average pooling
		pool2 = GlobalAveragePooling2D()(block)

		# dense
		fc1 = Dense(units=num_outputs,
					kernel_initializer='he_normal',
					activation='softmax')(pool2)

		return Model(inputs=input, outputs=fc1)

	@staticmethod
	def resnet_18(input_shape, num_outputs):
		return ResnetBuilder.build(
			input_shape, num_outputs, 'basic', [2, 2, 2, 2])

	@staticmethod
	def resnet_34(input_shape, num_outputs):
		return ResnetBuilder.build(
			input_shape, num_outputs, 'basic', [3, 4, 6, 3])

	@staticmethod
	def resnet_50(input_shape, num_outputs):
		return ResnetBuilder.build(
			input_shape, num_outputs, 'bottleneck', [3, 4, 6, 3])

	@staticmethod
	def resnet_101(input_shape, num_outputs):
		return ResnetBuilder.build(
			input_shape, num_outputs, 'bottleneck', [3, 4, 23, 3])

	@staticmethod
	def resnet_152(input_shape, num_outputs):
		return ResnetBuilder.build(
			input_shape, num_outputs, 'bottleneck', [3, 8, 36, 3])

Using TensorFlow backend.


## CIFAR10 データセットの用意

In [0]:
import keras
from keras.datasets import cifar10
import numpy as np

class CIFAR10Dataset():
	def __init__(self):
		self.image_shape = (32, 32, 3)
		self.num_classes = 10
		
	def preprocess(self, data, label_data=False):
		if label_data:
			# conver class number to one-hot vector
			data = keras.utils.to_categorical(data, self.num_classes)
		
		else:
			data = data.astype("float32")
			data /= 255 #convert the value to 0 ~ 1 scale
			shape = (data.shape[0],) + self.image_shape
			data = data.reshape(shape)
			
		return data
	
	def get_batch(self, subtract_pixel_mean=True):
		# x: data, y: lebel
		(x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Normalize data.
		x_train = x_train.astype('float32') / 255
		x_test = x_test.astype('float32') / 255
    
		if subtract_pixel_mean:
			x_train_mean = np.mean(x_train, axis=0)
			x_train -= x_train_mean
			x_test -= x_train_mean

		train_shape = (x_train.shape[0],) + self.image_shape
		x_train = x_train.reshape(train_shape)
    
		test_shape = (x_test.shape[0],) + self.image_shape
		x_test = x_test.reshape(test_shape)

    
		#x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
		y_train, y_test = [self.preprocess(d, label_data=True) for d in
					 [y_train, y_test]]

		
		return x_train, y_train, x_test, y_test

## TensorBoard用のログファイル生成関数

In [0]:
from __future__ import absolute_import
from __future__ import unicode_literals
from time import gmtime, strftime
from keras.callbacks import TensorBoard
import os


def make_tensorboard(set_dir_name=''):
    tictoc = strftime("%a_%d_%b_%Y_%H_%M_%S", gmtime())
    directory_name = tictoc
    log_dir = set_dir_name + '_' + directory_name
    os.mkdir(log_dir)
    tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, )
    return tensorboard

## ImageDataGeneratorクラスの拡張
random crop
mix up
cutout
を実装する。

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

class ImageDataGeneratorEX(ImageDataGenerator):
	def __init__(self,
               featurewise_center=False,
               samplewise_center=False, 
               featurewise_std_normalization=False,
               samplewise_std_normalization=False,
               zca_whitening=False,
               zca_epsilon=1e-06,
               rotation_range=0.0,
               width_shift_range=0.0,
               height_shift_range=0.0,
               brightness_range=None,
               shear_range=0.0,
               zoom_range=0.0, 
               channel_shift_range=0.0,
               fill_mode='nearest',
               cval=0.0,
               horizontal_flip=False, 
               vertical_flip=False,
               rescale=None,
               preprocessing_function=None,
               data_format=None,
               validation_split=0.0, 
               random_crop=None,    # a new parameter for random crop
               mix_up_alpha=0.0,    # a new parameter for mix up
               cutout_mask_size=0   # a new parameter for cutout
              ):
    
		# 親クラスのコンストラクタ
		super().__init__(featurewise_center=featurewise_center,
                     samplewise_center=samplewise_center,
                     featurewise_std_normalization=featurewise_std_normalization,
                     samplewise_std_normalization=samplewise_std_normalization,
                     zca_whitening=zca_whitening,
                     zca_epsilon=zca_epsilon,
                     rotation_range=rotation_range,
                     width_shift_range=width_shift_range,
                     height_shift_range=height_shift_range,
                     brightness_range=brightness_range,
                     shear_range=shear_range,
                     zoom_range=zoom_range,
                     channel_shift_range=channel_shift_range,
                     fill_mode=fill_mode,
                     cval=cval,
                     horizontal_flip=horizontal_flip,
                     vertical_flip=vertical_flip,
                     rescale=rescale,
                     preprocessing_function=preprocessing_function,
                     data_format=data_format,
                     validation_split=validation_split)
		
    # 拡張処理のパラメーター
		# Mix-up
		assert mix_up_alpha >= 0.0
		self.mix_up_alpha = mix_up_alpha
		# Random Crop
		assert random_crop == None or len(random_crop) == 2
		self.random_crop_size = random_crop
		self.cutout_mask_size = cutout_mask_size
    
	# ランダムクロップ
    # 参考 https://jkjung-avt.github.io/keras-image-cropping/
	def random_crop(self, original_img):
        # Note: image_data_format is 'channel_last'
		assert original_img.shape[2] == 3
		if original_img.shape[0] < self.random_crop_size[0] or original_img.shape[1] < self.random_crop_size[1]:
			raise ValueError(f"Invalid random_crop_size : original = {original_img.shape}, crop_size = {self.random_crop_size}")
		height, width = original_img.shape[0], original_img.shape[1]
		dy, dx = self.random_crop_size
		x = np.random.randint(0, width - dx + 1)
		y = np.random.randint(0, height - dy + 1)
		return original_img[y:(y+dy), x:(x+dx), :]

    # Mix-up
    # 参考 https://qiita.com/yu4u/items/70aa007346ec73b7ff05
	def mix_up(self, X1, y1, X2, y2):
		assert X1.shape[0] == y1.shape[0] == X2.shape[0] == y2.shape[0]
		batch_size = X1.shape[0]
		l = np.random.beta(self.mix_up_alpha, self.mix_up_alpha, batch_size)
		X_l = l.reshape(batch_size, 1, 1, 1)
		y_l = l.reshape(batch_size, 1)
		X = X1 * X_l + X2 * (1-X_l)
		y = y1 * y_l + y2 * (1-y_l)
		return X, y
    
	def cutout(self, x, y):
		return np.array(list(map(self._cutout, x))), y

	def _cutout(self, image_origin):
		# 最後に使うfill()は元の画像を書き換えるので、コピーしておく
		img = np.copy(image_origin)
		mask_value = img.mean()
		# 乱数固定(flowでseed固定したら必要ないかも)

		h, w, _ = img.shape
		# マスクをかける場所のtop, leftをランダムに決める
		# はみ出すことを許すので、0以上ではなく負の値もとる(最大mask_size // 2はみ出す)
		top = np.random.randint(0 - self.cutout_mask_size // 2, h - self.cutout_mask_size)
		left = np.random.randint(0 - self.cutout_mask_size // 2, w - self.cutout_mask_size)
		bottom = top + self.cutout_mask_size
		right = left + self.cutout_mask_size

		# はみ出した場合の処理
		if top < 0:
			top = 0
		if left < 0:
			left = 0

		# マスク部分の画素値を平均値で埋める
		img[top:bottom, left:right, :].fill(mask_value)
		return img


	def flow(self, 
			 x, y=None, 
			 batch_size=32, 
			 shuffle=True,
			 sample_weight=None,
			 seed=None, 
			 save_to_dir=None, 
			 save_prefix='', 
			 save_format='png', 
			 subset=None
		):
		
		batches = super().flow(x, y=y, 
                           batch_size=batch_size,
                           shuffle=shuffle,
                           sample_weight=sample_weight,
                           seed=seed,
                           save_to_dir=save_to_dir,
                           save_prefix=save_prefix,
                           save_format=save_format,
                           subset=subset)
    
		# 拡張処理
		while True:
			batch_x, batch_y = next(batches)
			
			# mix up
			if self.mix_up_alpha > 0:
				while True:
					batch_x_2, batch_y_2 = next(batches)
					m1, m2 = batch_x.shape[0], batch_x_2.shape[0]
					
					if m1 < m2:
						batch_x_2 = batch_x_2[:m1]
						batch_y_2 = batch_y_2[:m1]
						break
					elif m1 == m2:
						break
				batch_x, batch_y = self.mix_up(batch_x, batch_y, batch_x_2, batch_y_2)
			
			# Random crop
			if self.random_crop_size is not None:
				x = np.zeros((batch_x.shape[0], self.random_crop_size[0], self.random_crop_size[1], 3))
				for i in range(batch_x.shape[0]):
					x[i] = self.random_crop(batch_x[i])
				batch_x = x
			
			if self.cutout_mask_size > 0:
				batch_x, batch_y = self.cutout(batch_x, batch_y)
			
			
			yield (batch_x, batch_y)

## Training, Evaluation用クラスの定義

In [0]:
import os
from keras.models import load_model
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import RMSprop, Adam, Nadam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import numpy as np

from __future__ import absolute_import
from __future__ import unicode_literals
from time import gmtime, strftime
import os


def make_tensorboard(set_dir_name=''):
    tictoc = strftime("%a_%d_%b_%Y_%H_%M_%S", gmtime())
    directory_name = tictoc
    log_dir = set_dir_name + '_' + directory_name
    os.mkdir(log_dir)
    tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, )
    return tensorboard

class Trainer():
	
	def __init__(self, model, loss, optimizer, logdir = './'):
		self._target = model
		self._target.compile(
				loss=loss, optimizer=optimizer, metrics=["accuracy"]
				)
		self.verbose = 1 # visualize progress bar: 0(OFF), 1(On), 2(On:each data) 
		#self.log_dir = os.path.join(os.path.dirname(__file__), logdir)
		self.log_dir = os.path.join(logdir)
		self.model_file_name = "model_file.hdf5"

	def train_for_tuning_test_data(self, 
            x_train, y_train, x_test, y_test, batch_size, epochs, lr_scheduler):
		datagen = ImageDataGenerator(
			      featurewise_center=False,            # set input mean to 0 over the dataset
            samplewise_center=False,            # set each sample mean to 0
            featurewise_std_normalization=False, # divide inputs by std
            samplewise_std_normalization=False, # divide each input by its std
            zca_whitening=False,                 # apply ZCA whitening
            rotation_range=0,                   # randomly rotate images in the range (0~180)
            width_shift_range=4, #[-4,4]pixcel shift  # randomly shift images horizontally
            height_shift_range=4,              # randomly shift images vertically
            zoom_range = 0.0,
            channel_shift_range = 0.0,
            horizontal_flip=True,                # randomly flip images
            vertical_flip=False,                 # randomly flip images
            #random_crop=None,
			      #mix_up_alpha=0.0,
			      #cutout_mask_size=0
		)
    
    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
		#datagen.fit(x_train)
    
    # training (validation dataはデータ拡張はしない)
		model_path = os.path.join(self.log_dir, self.model_file_name)
		self._target.fit_generator(
            generator        = datagen.flow(x_train,y_train, batch_size),
            steps_per_epoch  = x_train.shape[0] // batch_size,
            epochs           = epochs,
            validation_data  = ImageDataGenerator().flow(x_test,y_test, batch_size),
			      validation_steps = x_test.shape[0] // batch_size,
            callbacks=[
                LearningRateScheduler(lr_scheduler),
                make_tensorboard(set_dir_name=self.log_dir),
                ModelCheckpoint(model_path, save_best_only=True,monitor='val_acc',mode='max')
            ],
            verbose = self.verbose,
            use_multiprocessing=True,
            #workers = 4
        )
		

class Evaluator():
    
    def __init__(self, result_file_path="./prediction_result.csv"):
        self.result_file_path="./prediction_result.csv"
        
    def simple_evaluate(self, model, x_test, label):
        print("start evaluation...")
        score = model.evaluate(x_test, y_test, verbose=1)
        print("Test loss:", score[0])
        print("Test accuracy:", score[1])
        return score
    
    def tta_evaluate(self, model, x_test, label, batch_size = 2500, tta_epochs = 2):
        print("batch size (TTA): "+str(batch_size))
        print("epochs (TTA): "+str(tta_epochs))
        tta = TTA()
        tta_pred = tta.predict(model, x_test, batch_size, epochs = tta_epochs)
        print("Test accuracy(TTA): ",end = "")
        print( accuracy_score( np.argmax(tta_pred,axis = 1) , np.argmax(label,axis = 1)))
        return tta_pred

## 学習率減衰

In [0]:
def learning_rate_schedule_for_Adam(epoch):
	lr = 0.001
	if(epoch >= 200): lr = 0.0002 
	if(epoch >= 275): lr = 0.00002
	return lr

def learning_rate_schedule_for_SGD(epoch):
	lr = 0.1
	if(epoch >=  60): lr /= 5 
	if(epoch >= 120): lr /= 5
	return lr

## 実行

In [0]:
from keras.optimizers import Adam, SGD
from keras.models import load_model


# create dataset
dataset = CIFAR10Dataset()
x_train, y_train, x_test, y_test = dataset.get_batch()


save_dir='/content/drive/My Drive/Colab Notebooks/Logs/ResNet_2/'

# create model
model = ResnetBuilder.resnet_34(dataset.image_shape, dataset.num_classes)
#model = ResnetBuilder.resnet_152(dataset.image_shape, dataset.num_classes)

#model = load_model(save_dir+'model_file.hdf5')
model.summary()

# train the model
#trainer = Trainer(model, loss="categorical_crossentropy", optimizer=SGD(lr=0.1, momentum=0.9, decay=0.0005, nesterov=True), logdir=save_dir)
trainer = Trainer(model, loss="categorical_crossentropy", optimizer=SGD(lr=0.1, decay=0.0005, momentum=0.9, nesterov=True), logdir=save_dir)
trainer.train_for_tuning_test_data(
            x_train, y_train, x_test, y_test, batch_size=256, epochs=200, 
            lr_scheduler=learning_rate_schedule_for_SGD)


# bestなモデルをロードする
model = load_model(save_dir+trainer.model_file_name)
#model = load_model(save_dir+"cnn_best_acc_model.hdf5")

# show result
evaluator = Evaluator()
score = evaluator.simple_evaluate(model, x_test, y_test)

W0826 18:12:58.670206 139761270921088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 18:12:58.684890 139761270921088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 18:12:58.690007 139761270921088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0826 18:12:58.722673 139761270921088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0826 18:12:58.723590

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 16, 16, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 16, 16, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 16, 16, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

W0826 18:13:04.324613 139761270921088 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0826 18:13:09.657641 139761270921088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0826 18:13:09.659255 139761270921088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/200
195/195 [==============================] - 52s 269ms/step - loss: 3.1167 - acc: 0.4829 - val_loss: 2.7966 - val_acc: 0.5793
Epoch 2/200
195/195 [==============================] - 45s 233ms/step - loss: 2.5857 - acc: 0.6332 - val_loss: 2.5908 - val_acc: 0.6306
Epoch 3/200
195/195 [==============================] - 45s 229ms/step - loss: 2.3428 - acc: 0.6919 - val_loss: 2.2978 - val_acc: 0.6964
Epoch 4/200
195/195 [==============================] - 45s 229ms/step - loss: 2.1687 - acc: 0.7277 - val_loss: 2.1899 - val_acc: 0.7083
Epoch 5/200
195/195 [==============================] - 45s 229ms/step - loss: 2.0330 - acc: 0.7519 - val_loss: 2.0599 - val_acc: 0.7368
Epoch 6/200
195/195 [==============================] - 45s 229ms/step - loss: 1.9240 - acc: 0.7700 - val_loss: 1.9799 - val_acc: 0.7435
Epoch 7/200
195/195 [==============================] - 45s 230ms/step - loss: 1.8248 - acc: 0.7872 - val_loss: 1.8823 - val_acc: 0.7630
Epoch 8/200
195/195 [===========================

##  Test Time Augmentation（TTA）を用いた推論 

In [0]:
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

class TTA:
    
    #test_time_augmentation
    #batch_sizeは，test_sizeの約数!!!
    def predict(self, model, x_test, batch_size ,epochs = 10):
        
        # Augmentation用generatorによるデータセットの作成
        data_flow = self.generator(x_test, batch_size)
        
        test_size = x_test.shape[0]
        pred = np.zeros(shape = (test_size,10), dtype = float)
        
        step_per_epoch = test_size //batch_size
        for epoch in range(epochs):
            print( 'epoch: ' + str(epoch+1)+'/'+str(epochs))
            for step in range(step_per_epoch):
                #print( 'step: ' + str(step+1)+'/'+str(step_per_epoch))
                sta = batch_size * step
                end = sta + batch_size
                tmp_x = data_flow.__next__()
                pred[sta:end] += model.predict(tmp_x)        
        return pred / epochs
    
    
    def generator(self, x_test,batch_size):
        return ImageDataGeneratorEX(
                    rotation_range = 10,
                    horizontal_flip = True,
                    height_shift_range = 0.1,
                    width_shift_range = 0.1,
                    zoom_range = 0.1,
                    channel_shift_range = 0.1,
            			  #random_crop=None,
			              #mix_up_alpha=0.2,
			              #cutout_mask_size=16
                ).flow(x_test,batch_size = batch_size,shuffle = False, seed=756) #756 9447

      
# show result
evaluator = Evaluator()
score = evaluator.tta_evaluate(model, x_test, y_test, batch_size = 500, tta_epochs = 50)

batch size (TTA): 500
epochs (TTA): 50
epoch: 1/50


ValueError: ignored